In [1]:
import os
import pandas as pd
import sys
import time

from PIL import Image
from sqlalchemy import create_engine, func, text
from sqlalchemy.orm import sessionmaker

sys.path.append('../..')
sys.path.append('../../../ajna_docs/commons')


In [2]:
from ajna_commons.flask.conf import SQL_URI
from ajna_commons.utils.images import mongo_image
from virasana.db import mongodb as db


2025-09-17 14:05 ajna         INFO     Configuração de log efetuada


2025-09-17 14:05:50,259 ajna         INFO     Configuração de log efetuada


In [3]:
%load_ext autoreload 
%autoreload 2
engine = create_engine(SQL_URI)

# Recuperar informações de REDEX e OPERADORES

In [4]:
# Constantes organizando códigos
LISTA_OPERADORES = "('8931356', '8931359', '8931404', '8931318')"  # Terminais exportadores BTP, SANTOSBRASIL, DPW/EMBRAPORT e ECOPORTO
LISTA_REDEX = "('8931309', '8933204', '8931404')" # Redex da Localfrio/Movecta, Redex da Santos Brasil Clia e Redex da DPW/EMBRAPORT
FILTRO_LIKE_REDEX = "'89327%'"  # Abrange todos os outros Redex santistas

from datetime import datetime, timedelta

agora = datetime.now()
ha_X_dias = agora - timedelta(days=7)

# Parâmetros de data
data_final = agora.strftime('%Y-%m-%d %H:%M:%S')
data_inicial = ha_X_dias.strftime('%Y-%m-%d %H:%M:%S')

In [5]:
# SQL para entrada_operadores (alias e)
sql_entrada_operadores = f"""
SELECT 
    id, placa, placaSemirreboque, numeroConteiner, codigoRecinto, dataHoraOcorrencia, operacao, direcao, 
    cnpjTransportador, cpfMotorista, nomeMotorista, listaNfe, numeroConhecimento
FROM apirecintos_acessosveiculo
WHERE 
    dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
    AND operacao = 'C'
    AND direcao = 'E'
    AND codigoRecinto IN {LISTA_OPERADORES}
    AND numeroConteiner !='' 
"""

# Este símbolo != '' Significa que o numero do conteiner NÃO pode ser nulo porque queremos caminão com conteiner
# Este 'E' é porque queremos a Entrada no Operador Terminal Exportador (BTP, SANTOSBRASIL, DPW/EMBRAPORT e ECOPORTO)
# Este 'C' é porque é registro de acesso (entrada) e não agendamento

In [6]:
# SQL para pesagem_operador - pesagem com recintos dos operadores (terminais exportadores)
sql_pesagem_operador = f"""
SELECT 
    id, placa, placaSemirreboque, pesoBrutoBalanca, numeroConteiner, codigoRecinto, dataHoraOcorrencia
FROM apirecintos_pesagensveiculo
WHERE 
    pesoBrutoBalanca >= 22000
    AND dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
    AND codigoRecinto IN {LISTA_OPERADORES}
"""
# Se o caminhao com conteiner pesa 22 toneladas ou mais, então, ele está carregado (não é vazio)

In [7]:
# SQL para pesagem_redex - pesagem com recintos Redex (estufadores)
sql_pesagem_redex = f"""
SELECT 
    id, placa, placaSemirreboque, pesoBrutoBalanca, numeroConteiner, codigoRecinto, dataHoraOcorrencia
FROM apirecintos_pesagensveiculo
WHERE 
    pesoBrutoBalanca >= 22000
    AND dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
    AND (codigoRecinto IN {LISTA_REDEX} OR codigoRecinto LIKE {FILTRO_LIKE_REDEX})
"""
# Se o caminhao com conteiner pesa 22 toneladas ou mais, então, ele está carregado (não é vazio)

In [8]:
# SQL para saida_redex (alias c)
sql_saida_redex = f"""
SELECT 
    id, placa, placaSemirreboque, numeroConteiner, codigoRecinto, cpfMotorista, nomeMotorista, dataHoraOcorrencia, direcao
FROM apirecintos_acessosveiculo
WHERE
    dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
    AND operacao = 'C'
    AND direcao = 'S'
    AND (codigoRecinto IN {LISTA_REDEX} OR codigoRecinto LIKE {FILTRO_LIKE_REDEX})
    AND numeroConteiner !=''
"""
# Este símbolo != '' Significa que o numero do conteiner NÃO pode ser nulo porque queremos caminão com conteiner
# Este 'S' é porque queremos a Saida do Redex estufador
# Este 'C' é porque é registro de acesso (saída) e não agendamento

In [9]:
# Execução das consultas com parâmetros
df_entrada_operadores = pd.read_sql(
    text(sql_entrada_operadores), engine, params={"data_inicial": data_inicial, "data_final": data_final}, parse_dates=['dataHoraOcorrencia']
)

In [10]:
df_entrada_operadores

,id,placa,placaSemirreboque,numeroConteiner,codigoRecinto,dataHoraOcorrencia,operacao,direcao,cnpjTransportador,cpfMotorista,nomeMotorista,listaNfe,numeroConhecimento
0,12247108,FRS2D82,AIK5928,HLBU9318070,8931318,2025-09-10 14:12:15,C,E,12014047000193,11945072806,MARCOS CESAR DE MENDONCA,,152505266678571
1,12247109,BYG8B56,TMI8F90,MSMU7918550,8931318,2025-09-10 14:17:24,C,E,01896217000194,30370113861,FABRICIO DEL ALAMO MISTURA,,152505253759179
2,12247112,AUX5B91,ENN0D25,MSNU7294329,8931318,2025-09-10 14:28:12,C,E,01896217000194,29657468892,RODRIGO DO NASCIMENTO SILVA,,152505254470121
3,12247115,GMH5771,BXJ7G84,MSMU4411639,8931318,2025-09-10 14:30:00,C,E,32256956000145,47573945434,EVALDO RODRIGUES DA SILVA,,
4,12247113,DBM9G64,EKS1A42,GCXU2373813,8931318,2025-09-10 14:30:29,C,E,21425093000176,38746047808,ERIK VICTOR CRUZ DE OLIVEIRA,,152505267204999
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26818,12435875,BTS6B31,SSD0215,MRSU4911320,8931404,2025-09-16 23:35:13,C,E,49871560000100,98519492568,ANTONIO CARLOS DE JESUS ANDRAD,35250906538082000109550060000473461000038928,
26819,12435890,EHH8D36,FGX8H83,CAAU6473503,8931404,2025-09-16 23:43:29,C,E,09621639000169,31194790801,EDUARDO APARECIDO NUNES,31250916849231001003550010000407551142681849,
26820,12435907,QBO8A00,DPS3F60,MRKU4371066,8931404,2025-09-16 23:56:12,C,E,02427026000146,31419975862,FABIO HENRIQUE DE MORAES KREBS,35250911931402000126550060001278691004544452,
26821,12435905,SSS3J26,BJX9J69,MNBU3005822,8931404,2025-09-16 23:56:20,C,E,02916265009116,33750239878,TIAGO LUIZ DA SILVA,35250902914460018793550010001183651929255269,


In [11]:
df_saida_redex = pd.read_sql(
    text(sql_saida_redex), engine, params={"data_inicial": data_inicial, "data_final": data_final}, parse_dates=['dataHoraOcorrencia']
)

In [12]:
df_saida_redex

,id,placa,placaSemirreboque,numeroConteiner,codigoRecinto,cpfMotorista,nomeMotorista,dataHoraOcorrencia,direcao
0,12434374,CUK4G26,DPE6975,HLBU9357312,8931404,33445491895,RENATO CAVALCANTE CAMARA,2025-09-16 08:21:44,S
1,12434376,TKK4E35,SVG0B81,ZCSU7800026,8931404,35142434846,JOSE RAFAEL DE JESUS,2025-09-16 08:23:07,S
2,12434377,PPJ7G44,DPE9A82,TCNU2439934,8931404,02824372664,AILSON LISBOA DE MATOS,2025-09-16 08:23:40,S
3,12434379,FDY0919,FEI4091,HOYU9660407,8931404,32933527820,LEONARDO DE CARVALHO PINTO MAR,2025-09-16 08:23:50,S
4,12434384,CNR1J79,BSF4F24,HMMU5434910,8931404,35177073840,SIDNEY DO NASCIMENTO CHAVES,2025-09-16 08:29:42,S
...,...,...,...,...,...,...,...,...,...
3643,12438701,NTO9D51,BXE5B09,HLBU3603810,8932798,39914451829,GLEIDSON JUNIOR SANTOS,2025-09-16 21:24:41,S
3644,12438703,AER6J30,DVS1H39,TIIU5303730,8932798,27298922815,CELSO MARQUES HANZOI,2025-09-16 21:26:18,S
3645,12438652,INX8J19,DPF0B10,DRYU6088584,8932798,25278514876,FABIO MOREIRA SANTOS,2025-09-16 21:27:54,S
3646,12438704,BMT8G91,BTR1527,FFAU5354870,8932798,33273236825,IVANILSON CAVALCANTE MARINHO,2025-09-16 21:32:11,S


In [13]:
df_saidas_entradas = pd.merge(df_entrada_operadores, df_saida_redex, how='inner', on=['placaSemirreboque', 'numeroConteiner'] , suffixes=('_E_Operador', '_S_REDEX'))
# Este 'inner' faz com que a tabela final obtida saidas_entradas só tenham dados que tem em Entrada Operador e Saída Rede
# Dados que não tenham ligação correspondente (são nulos) NÃO aparecerão
# Se você quiser que os não correspondentes também apareçam troque INNER por LEFT

In [14]:
df_saidas_entradas['transitTime'] = (df_saidas_entradas['dataHoraOcorrencia_E_Operador'] - 
                                     df_saidas_entradas['dataHoraOcorrencia_S_REDEX']).dt.total_seconds() / 3600


In [15]:
# df_saidas_entradas

In [16]:
#!pip install tqdm
from tqdm.notebook import tqdm

def buscar_id_fs_files(collection, numero_conteiner, data_ocorrencia):
    filtro = {
        'metadata.numeroinformado': numero_conteiner,
        'metadata.dataescaneamento': {
            '$gte': data_ocorrencia - timedelta(hours=3),
            '$lte': data_ocorrencia + timedelta(hours=6)
        }
    }
    resultado = collection.find_one(filtro, {'_id': 1})
    if resultado:
        return resultado['_id']
    return None

# Iterar sobre as linhas do DataFrame e aplicar busca
def mapear_ids_mongo(collection, df):
    ids_fs_files = []
    cont_sucesso = 0
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Buscando ids no MongoDB"):
        numero = row['numeroConteiner']
        data = row['dataHoraOcorrencia_E_Operador']
        id_fs = buscar_id_fs_files(collection, numero, data)
        ids_fs_files.append(id_fs)
        cont_sucesso += 1 if id_fs else 0
    df['fs_files_id'] = ids_fs_files
    print(f'Registros: {len(df)}, imagens: {cont_sucesso}')

In [17]:
mapear_ids_mongo(db['fs.files'], df_saidas_entradas)

Buscando ids no MongoDB:   0%|          | 0/2107 [00:00<?, ?it/s]

Registros: 2107, imagens: 878


In [18]:
# df_saidas_entradas.transitTime.describe()
df_saidas_entradas.rename(columns={'placa': 'placa', 'placaSemirreboque': 'placa do semireboque', 'numeroConteiner': 'contêiner',
                         'codigoRecinto_S_REDEX': 'Redex', 'dataHoraOcorrencia_S_REDEX': 'Saída REDEX','placaSemirreboque': 'placa do semireboque',
                         'codigoRecinto_E_Operador': 'Operador', 'dataHoraOcorrencia_E_Operador': 'Entrada Operador',
                         'transitTime': 'transitTime', 'fs_files_id': '_id'}, inplace=True)
df_saidas_entradas.drop(['operacao', 'direcao_E_Operador', 'listaNfe', 'numeroConhecimento', 'direcao_S_REDEX'], axis=1)
df_saidas_entradas

,id_E_Operador,placa_E_Operador,placa do semireboque,contêiner,Operador,Entrada Operador,operacao,direcao_E_Operador,cnpjTransportador,cpfMotorista_E_Operador,...,numeroConhecimento,id_S_REDEX,placa_S_REDEX,Redex,cpfMotorista_S_REDEX,nomeMotorista_S_REDEX,Saída REDEX,direcao_S_REDEX,transitTime,_id
0,12423418,MTC4J24,FAX1317,HMMU6543221,8931318,2025-09-16 09:34:07,C,E,12014047000193,31704711800,...,152505273544020,12434386,MTC4J24,8931404,31704711800,JOSE ADILSON BISPO DOS SANTOS,2025-09-16 08:30:51,S,1.054444,68c97640f614e4f4040efa19
1,12423435,CUD4902,FOE4388,HMMU6790052,8931318,2025-09-16 09:56:49,C,E,12014047000193,10430518803,...,,12434419,CUD4902,8931404,10430518803,ALDO PASCOAL SOARES JUNIOR,2025-09-16 08:53:56,S,1.048056,68c97640f614e4f4040efa1e
2,12423469,EWJ9661,FTU5124,KOCU4524804,8931318,2025-09-16 11:19:54,C,E,12014047000193,90643925449,...,,12434555,EWJ9661,8931404,90643925449,ANDRE TEIXEIRA DA SILVA SANTOS,2025-09-16 10:16:34,S,1.055556,68ca98be4b6fcba5bbb13a1d
3,12423491,FEI3I38,FUY9846,HMMU5519489,8931318,2025-09-16 11:54:34,C,E,12014047000193,17898192885,...,152505262479639,12434571,FEI3I38,8931404,17898192885,ANTONIO CARLOS BRAGA,2025-09-16 10:22:29,S,1.534722,68ca98be4b6fcba5bbb13a18
4,12423598,KJQ0F31,FUI4356,MOFU6706340,8931318,2025-09-16 17:01:37,C,E,12014047000193,08923508810,...,152505261574760,12435152,KJQ0F31,8931404,08923508810,PEDRO DE SOUZA,2025-09-16 16:02:10,S,0.990833,68ca990e8e8f95cb27bf8bac
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2102,12435815,AER6J30,DVS1H39,TIIU5303730,8931404,2025-09-16 22:55:51,C,E,58130089000786,27298922815,...,,12438703,AER6J30,8932798,27298922815,CELSO MARQUES HANZOI,2025-09-16 21:26:18,S,1.492500,None
2103,12435819,ASX1H52,LUW9A79,ONEU5800240,8931404,2025-09-16 22:57:19,C,E,58130089000786,00113214588,...,,12438698,ASX1H52,8932798,00113214588,JORGE CASCIO SOUZA SANTOS SANTANA,2025-09-16 21:09:49,S,1.791667,None
2104,12435836,LQF3D34,FEI3E98,GCXU5606987,8931404,2025-09-16 23:03:24,C,E,58130089000786,43713671869,...,,12438635,LQF3D34,8932798,43713671869,MARCOS VINICIUS GOMES DE MORAIS,2025-09-16 21:34:40,S,1.478889,None
2105,12435840,NTO9D51,BXE5B09,HLBU3603810,8931404,2025-09-16 23:07:46,C,E,58130089000786,39914451829,...,,12438701,NTO9D51,8932798,39914451829,GLEIDSON JUNIOR SANTOS,2025-09-16 21:24:41,S,1.718056,None


In [19]:
from IPython.display import display, HTML

def get_outliers(df):
    mean = df['transitTime'].mean()
    std = df['transitTime'].std()
    outliers = df[(df['transitTime'] < mean - 1 * std) | (df['transitTime'] > mean + 1 * std)]
    return outliers

def generate_html(_id):
    url = f"https://ajna1.rfoc.srf/virasana/file?_id={_id}"
    img = f"https://ajna1.rfoc.srf/virasana/imagens_cmap?_id={_id}"
    return f'<a href="{url}"><img src="{img}" width="200"></a>'
    
outliers = get_outliers(df_saidas_entradas)[['placa do semireboque', 'contêiner', 'Redex', 'Saída REDEX',
                                  'Operador', 'Entrada Operador', 'transitTime', '_id']]

outliers['html'] = outliers['_id'].apply(generate_html)
display(HTML(outliers.to_html(escape=False)))

,placa do semireboque,contêiner,Redex,Saída REDEX,Operador,Entrada Operador,transitTime,_id,html
268,DBB7G03,HLXU8202041,8932796,2025-09-10 16:22:31,8931356,2025-09-11 13:58:19,21.596667,None,
269,DBB7G03,HLXU8202041,8932796,2025-09-10 16:47:45,8931356,2025-09-11 13:58:19,21.176111,None,
316,TIQ7A51,FANU1401220,8932775,2025-09-11 09:13:17,8931356,2025-09-11 18:11:24,8.968611,None,
324,TIT0E61,TGBU9609122,8932775,2025-09-10 23:22:12,8931356,2025-09-11 20:55:31,21.555278,None,
536,CUA6448,CAAU5948796,8932722,2025-09-10 17:33:01,8931356,2025-09-12 14:25:57,44.882222,None,
601,FKS0890,FSCU5720867,8932722,2025-09-11 18:50:45,8931356,2025-09-12 18:12:06,23.355833,None,
602,CUA7446,ONEU9230398,8932722,2025-09-11 18:57:06,8931356,2025-09-12 18:12:16,23.252778,None,
655,FKS0894,ONEU9386251,8932722,2025-09-12 02:30:48,8931356,2025-09-13 01:18:59,22.803056,None,
657,BSF4626,TTNU8791626,8932722,2025-09-11 18:32:48,8931356,2025-09-13 01:19:54,30.785000,None,
798,CUA5121,TCKU6215898,8932796,2025-09-13 13:43:08,8931356,2025-09-15 23:33:31,57.839722,None,


In [20]:
# Adicionar células clicáveis que levem ao Ajna
df_saidas_entradas['contêiner'] = df_saidas_entradas['contêiner'].apply(
    lambda x: f'=HYPERLINK("https://ajna1.rfoc.srf/bhadrasana2/consulta_container?numerolote={x}", "{x}")'
)
df_saidas_entradas['_id'] = df_saidas_entradas['_id'].apply(
    lambda x: f'=HYPERLINK("https://ajna1.rfoc.srf/virasana/file?_id={x}", "{x}")'
)

# df_saidas_entradas

In [21]:
df_saidas_entradas.to_excel('saidas_entradas.xlsx', index=False)

In [22]:
df_pesagem_operador = pd.read_sql(
    text(sql_pesagem_operador), engine, params={"data_inicial": data_inicial, "data_final": data_final}, parse_dates=['dataHoraOcorrencia']
)

df_pesagem_redex = pd.read_sql(
    text(sql_pesagem_redex), engine, params={"data_inicial": data_inicial, "data_final": data_final}, parse_dates=['dataHoraOcorrencia']
)



# Seguem os merges e filtros conforme a lógica desejada especificada anteriormente
# Por exemplo, aqui unimos pesagem_redex com saida_redex e depois com entrada_operadores conforme passo a passo
df_pesagem_saida = pd.merge(df_pesagem_redex, df_saida_redex, on=['placaSemirreboque', 'codigoRecinto', 'numeroConteiner'], suffixes=('_pesagem', '_saida'))

df_pesagem_saida = df_pesagem_saida[
    (df_pesagem_saida['dataHoraOcorrencia_pesagem'] <= df_pesagem_saida['dataHoraOcorrencia_saida']) &
    ((df_pesagem_saida['dataHoraOcorrencia_saida'] - df_pesagem_saida['dataHoraOcorrencia_pesagem']).dt.days <= 7) &
    (df_pesagem_saida['direcao'] == 'S')
]

df_final = pd.merge(df_pesagem_saida, df_entrada_operadores, left_on=['placaSemirreboque', 'numeroConteiner'], right_on=['placaSemirreboque', 'numeroConteiner'], suffixes=('', '_entrada'))

df_final = df_final[
    ((df_final['dataHoraOcorrencia'] - df_final['dataHoraOcorrencia_saida']).dt.days <= 7) &
    (df_final['codigoRecinto'] != df_final['codigoRecinto_entrada']) 
]

df_final = df_final.drop_duplicates()

# Seleção de colunas para output
output_cols = [
    'dataHoraOcorrencia_pesagem', 'id_pesagem', 'placa', 'placaSemirreboque', 'pesoBrutoBalanca', 'numeroConteiner', 'codigoRecinto_pesagem',
    'dataHoraOcorrencia_saida', 'placa', 'codigoRecinto_saida', 'direcao', 'numeroConteiner',
    'dataHoraOcorrencia', 'placa', 'placaSemirreboque', 'id', 'cnpjTransportador', 'cpfMotorista', 'nomeMotorista', 'codigoRecinto', 'operacao', 'direcao', 'numeroConteiner', 'listaNfe', 'numeroConhecimento'
]

df_result = df_final[output_cols]

print(df_result.head())

KeyError: "['codigoRecinto_pesagem', 'codigoRecinto_saida'] not in index"